# Error Handling in Python - A Comprehensive Tutorial

This notebook explores various error handling techniques in Python, focusing on:
1. Basic exception handling
2. Creating custom exceptions
3. Exception propagation
4. Logging errors
5. Best practices and common patterns

Let's start by importing the necessary modules:

In [1]:
from utility.log_manager import setup_logging

logger = setup_logging()

## 1. Basic Exception Handling

Let's start with a simple example of try-except blocks:

In [1]:
10 / 0

ZeroDivisionError: division by zero

In [26]:
try:
    x = 1
    y = "2"
    z = x / y
except ZeroDivisionError:
    print("You can't divide by zero!")
except TypeError as e:
    raise TypeError("You can't divide by a string!: " + str(e))

TypeError: You can't divide by a string!: unsupported operand type(s) for /: 'int' and 'str'

In [29]:
def a():
    try:
        x = 1
        y = "2"
        z = x / y
    except ZeroDivisionError:
        print("You can't divide by zero!")
    except TypeError:
        print("Wrong type!")


def b():
    try:
        a()
    except ZeroDivisionError:
        print("You can't divide by zero!")
    except TypeError:
        print("Error: Wrong type!")


b()

# If you log a message inside an except block, a message will be logged every time the exception is raised.
# If you type 'raise' in an except block, the exception will be outputted
# If you re-raise the exception "raise Exception", the original exception, and new exception will be outputted

TypeError: unsupported operand type(s) for /: 'int' and 'str'

In [3]:
def divide(a, b):
    try:
        result = a / b
        return result
    except ZeroDivisionError:
        print("Error: Division by zero!")
        return None


print(divide(10, 2))  # Normal case

5.0


In [4]:
print(divide(10, 0))  # Division by zero

Error: Division by zero!
None


In [7]:
try:
    open("nofile.txt")
except OSError:
    raise RuntimeError("unable to handle error")

# try:
#    open("nofile.txt")
# except OSError as exc:
#    raise RuntimeError from exc

# try:
#    open("nofile.txt")
# except OSError as exc:
#    raise RuntimeError from None

RuntimeError: unable to handle error

In [9]:
try:
    try:
        raise ValueError("ValueError1")
    except ValueError as e1:
        raise TypeError("TypeError2") from e1
except TypeError as e2:
    print("The exception was", repr(e2))
    print("Its __context__ was", repr(e2.__context__))
    print("Its __cause__ was", repr(e2.__cause__))

The exception was TypeError('TypeError2')
Its __context__ was ValueError('ValueError1')
Its __cause__ was ValueError('ValueError1')


## 2. Creating Custom Exceptions

Now, let's create a custom exception hierarchy:

In [9]:
class DataProcessingError(Exception):
    """Base exception for data processing errors"""

    pass


class InvalidDataError(DataProcessingError):
    """Raised when the input data is invalid"""

    pass


class DataNotFoundError(DataProcessingError):
    """Raised when the required data is not found"""

    pass


def process_data(data):
    if not data:
        raise DataNotFoundError("No data provided")
    if not isinstance(data, list):
        raise InvalidDataError("Data must be a list")
    return sum(data)


try:
    result = process_data([1, 2, 3])
    print(f"Result: {result}")
    result = process_data({})
except DataProcessingError as e:
    print(f"Error processing data: {e}")

Result: 6
Error processing data: No data provided


## 3. Exception Propagation

Let's explore how exceptions propagate through function calls:

In [8]:
def inner_function(x):
    if x < 0:
        raise ValueError("x must be non-negative")
    return x**0.5


def outer_function(x):
    try:
        return inner_function(x)
    except ValueError as e:
        print(f"Caught in outer_function: {e}")
        raise  # Re-raise the exception


try:
    result = outer_function(-5)  # Negative input
except ValueError as e:
    print(f"Caught in main: {e}")

Caught in outer_function: x must be non-negative
Caught in main: x must be non-negative


## 4. Logging Errors

Now, let's incorporate logging into our error handling:

In [10]:
def risky_operation(data):
    try:
        if len(data) < 2:
            raise ValueError("Data must have at least two elements")
        return data[0] / data[1]
    except IndexError as e:
        logger.error(f"IndexError in risky_operation: {e}")
        raise
    except ZeroDivisionError as e:
        logger.warning(f"ZeroDivisionError in risky_operation: {e}")
        return float("inf")
    except Exception as e:
        logger.critical(f"Unexpected error in risky_operation: {e}")
        raise


try:
    print(risky_operation([10, 2]))  # Normal case
    print(risky_operation([10, 0]))  # Division by zero
    print(risky_operation([]))  # IndexError
except Exception as e:
    print(f"Caught in main: {type(e).__name__}: {e}")

[2024-07-04 16:08:38] [WARNING]  ZeroDivisionError in risky_operation: division by zero
[2024-07-04 16:08:38] [CRITICAL]  Unexpected error in risky_operation: Data must have at least two elements


5.0
inf
Caught in main: ValueError: Data must have at least two elements


## 5. Best Practices and Common Patterns

Let's explore some best practices and common patterns in error handling:

In [11]:
class APIError(Exception):
    """Base exception for API errors"""

    pass


class NetworkError(APIError):
    """Raised when a network operation fails"""

    pass


class DataParsingError(APIError):
    """Raised when parsing API response fails"""

    pass


def api_request(endpoint):
    # Simulating an API request
    if endpoint == "error":
        raise NetworkError("Connection failed")
    return {"data": "some data"}


def parse_response(response):
    # Simulating response parsing
    if "error" in response:
        raise DataParsingError("Invalid response format")
    return response["data"]


def get_api_data(endpoint):
    try:
        response = api_request(endpoint)
        data = parse_response(response)
        return data
    except NetworkError as e:
        logger.error(f"Network error occurred: {e}")
        raise APIError("Failed to connect to API") from e
    except DataParsingError as e:
        logger.error(f"Data parsing error occurred: {e}")
        raise APIError("Failed to parse API response") from e
    except Exception as e:
        logger.critical(f"Unexpected error in get_api_data: {e}")
        raise APIError("An unexpected error occurred") from e


try:
    data = get_api_data("valid_endpoint")
    print(f"Received data: {data}")
    data = get_api_data("error")
except APIError as e:
    print(f"API Error: {e}")
    if e.__cause__:
        print(f"Caused by: {type(e.__cause__).__name__}: {e.__cause__}")

[2024-07-04 16:12:22] [ERROR]  Network error occurred: Connection failed


Received data: some data
API Error: Failed to connect to API
Caused by: NetworkError: Connection failed


## Conclusion

This tutorial covered various aspects of error handling in Python:
1. We started with basic try-except blocks.
2. We created custom exception hierarchies for more specific error handling.
3. We explored exception propagation through function calls.
4. We incorporated logging to provide more context for errors.
5. Finally, we looked at best practices and common patterns in error handling.

Remember these key points:
- Use specific exceptions when possible.
- Create custom exceptions for your application's specific needs.
- Log errors with appropriate severity levels.
- Use `raise ... from e` to preserve the original error context.
- Handle exceptions at the appropriate level in your application.

Proper error handling makes your code more robust, easier to debug, and provides a better experience for users of your application.